In [1]:
from __future__ import division
import pandas as pd
import glob
import os
from datetime import datetime
import pickle
from PIL import Image
from VAMPIRE_backend import cntarea,cntAR,cntExtent,cntEquiDia,cntsol,bdreg,pca_bdreg,clusterSM,update_csv
from time import time
import numpy as np
from matplotlib import pyplot as plt

In [2]:
N=50 #equidistant points along cell boundary
clnum = 10
modelname='testmodel4'
outpth = r'C:\Users\kyuha\PycharmProjects\VAMPIRE_open\Source Code Contour\new code'

In [3]:
modelpth = r'C:\Users\kyuha\PycharmProjects\VAMPIRE_open\Source Code Contour\new code\{}.pickle'.format(modelname)
tilepth = r'\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE'
setpaths = glob.glob(os.path.join(tilepth,'*'))
setID = [os.path.basename(_).split('_')[1] for _ in setpaths] #patient id
condition = [os.path.basename(_).split('_')[3] for _ in setpaths] #section id
setpaths = [os.path.join(*[_,'out','json']) for _ in setpaths]

In [4]:
def cellclass(cnt):
    celltype = dlim.getpixel((round(cnt[0]),round(cnt[1])))
    return celltype

In [5]:
ttt = Image.open(r'\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\DLTILE20X\imID_205_sec_3\317.tif')

UnidentifiedImageError: cannot identify image file '\\\\fatherserverdw\\Q\\research\\images\\skin_aging\\hovernet_tile\\DLTILE20X\\imID_205_sec_3\\317.tif'

In [8]:
jsnucs = pd.DataFrame()
imuniques=[]
for setidx, setpath in enumerate(setpaths[200:300]):
    # patientdir = r'\\motherserverdw\Kyu_Sync\Research\Database\Hamamatsu Nanozoomer Temporary Repository\IMCROP20x\IMCROP_processed\TILE\imID_8_sec_1'
    # jsondir = os.path.join(*[patientdir,'out','json'])
    start = time()
    jsondir = setpath
    # if os.path.exists(os.path.join(jsondir,'boundary.pkl')):continue
    jsons = glob.glob(os.path.join(jsondir,'*json'))
    dldir = os.path.dirname(os.path.dirname(jsondir)).replace('TILE','DLTILE20X')
    # dls = glob.glob(os.path.join(dldir,'*tif'))
    jsnucs = pd.DataFrame()
    if len(jsons)==0:
        print(setidx,'no json files detected')
        continue
    for js in jsons:
        jss = pd.read_json(js)
        imid = os.path.splitext(os.path.basename(js))[0]
        dl = os.path.join(dldir,imid+'.tif')
        try:
            dlim = Image.open(dl)
        except:
            print('failed to open DL image: ', dl)
            continue
        try:
            imuniques.append(np.max(np.unique(dlim)))
        except:
            imuniques.append(np.max(np.unique(dlim)))
        jsnuc = jss['nuc'].apply(pd.Series)
        if len(jsnuc)==0:continue
        jsnuc = jsnuc.drop(['type_prob'], axis=1).reset_index(drop=True)
        jsnuc['celltype'] = jsnuc['centroid'].apply(lambda row: cellclass(row))
        jsnuc = jsnuc.drop(['bbox','centroid','type'], axis=1)
        dlim.close()
        jsnucs = pd.concat([jsnucs,jsnuc],ignore_index=True)
    jsnucs = jsnucs[jsnucs['contour'].map(len) > 5]
    jsnucs['Area'] = jsnucs['contour'].apply(lambda row: cntarea(row))
    jsnucs['AR'] = jsnucs['contour'].apply(lambda row: cntAR(row))
    jsnucs['Sol'] = jsnucs['contour'].apply(lambda row: cntsol(row))
    jsnucs['Extent'] = jsnucs['contour'].apply(lambda row: cntExtent(row))
    jsnucs['EquiDia'] = jsnucs['contour'].apply(lambda row: cntEquiDia(row))
    jsnucs.to_pickle(os.path.join(jsondir,'boundary.pkl'))
    print(os.path.join(jsondir,'boundary.pkl'))
    print(setidx,round(time()-start),'seconds')

\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_240_sec_3\out\json\boundary.pkl
0 17 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_240_sec_4\out\json\boundary.pkl
1 14 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_241_sec_1\out\json\boundary.pkl
2 22 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_241_sec_2\out\json\boundary.pkl
3 15 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_241_sec_3\out\json\boundary.pkl
4 22 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_241_sec_4\out\json\boundary.pkl
5 16 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_243_sec_1\out\json\boundary.pkl
6 6 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_243_sec_2\out\json\boundary.pkl
7 7 seconds
\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\TILE\imID_243_sec_3\ou

In [ ]:
start = time()
pkls = []
for setpath in setpaths:
    start = time()
    if not os.path.exists(os.path.join(setpath,'boundary.pkl')):continue
    pkl = pd.read_pickle(os.path.join(setpath,'boundary.pkl'))
    pkls.append(pkl)
pkls = pd.concat(pkls,ignore_index=True)
print(round(time()-start),'seconds')
pkls

In [ ]:
np.unique(pkls['celltype'])

In [ ]:
# count cell population distribution by class
lens=[]
for i in range(13):
    lens.append(len(pkls[pkls['celltype']==i]))
lens

In [ ]:
# visualize cell distribution
start = time()
a=plt.hist(pkls['celltype'])
print(round(time()-start),'seconds')
a

In [ ]:
tags = ['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','blood','ECM','fat']
for idx,tag in enumerate(tags):
    modelname = tag
    bd1 = pkls[pkls['celltype']==idx+1].reset_index()
    B=bd1['contour']
    VamModel = {
                "N": [],
                "bdrn": [],
                "mdd": [],
                "pc": [],
                "clnum": [],
                "pcnum": [],
                "mincms": [],
                "testmean": [],
                "teststd": [],
                "boxcoxlambda": [],
                "C": [],
                "Z": []
            }

    build_model = True
    bdpc, VamModel = bdreg(B, N, VamModel, build_model)
    score, VamModel = pca_bdreg(bdpc, VamModel, True)
    pcnum = None # none is 20 by default
    IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)


    if os.path.exists(os.path.join(*[outpth, modelname, modelname+'.pickle'])):
        realtimedate = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        f = open(os.path.join(*[outpth, modelname, modelname+'_'+realtimedate+'.pickle']), 'wb')
    else:
        f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
    pickle.dump(VamModel, f)
    f.close()


In [ ]:
modelname = 'epidermis'
bd1 = pkls[pkls['celltype']==3].reset_index()
B=bd1['contour']
VamModel = {
            "N": [],
            "bdrn": [],
            "mdd": [],
            "pc": [],
            "clnum": [],
            "pcnum": [],
            "mincms": [],
            "testmean": [],
            "teststd": [],
            "boxcoxlambda": [],
            "C": [],
            "Z": []
        }

build_model = True
bdpc, VamModel = bdreg(B, N, VamModel, build_model)
score, VamModel = pca_bdreg(bdpc, VamModel, True)
pcnum = None # none is 20 by default
IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)


if os.path.exists(os.path.join(*[outpth, modelname, modelname+'.pickle'])):
    realtimedate = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    f = open(os.path.join(*[outpth, modelname, modelname+'_'+realtimedate+'.pickle']), 'wb')
else:
    f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
pickle.dump(VamModel, f)
f.close()

In [ ]:
experimental = False

In [ ]:
try:
    f = open(modelpth, 'rb')
except:
    print('the model does not exist. please replace model name to the one you built')
vampire_model = pickle.load(f)
N = vampire_model['N']
clnum = vampire_model['clnum']
pcnum = vampire_model['pcnum']

In [ ]:
for setidx, setpath in enumerate(setpaths):
    try:
        print(setidx,setpath)
        pkl = pd.read_pickle(os.path.join(setpath,'boundary.pkl'))
        B = pkl['contour']
        pkl = pkl.drop(['bbox','centroid','contour','type'], axis=1)
        bdpc, vampire_model = bdreg(B, N, vampire_model, build_model)
        score, vampire_model = pca_bdreg(bdpc, vampire_model, build_model)
        IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, vampire_model, False, condition[setidx], setID[setidx],modelname)

        datasheet = 'VAMPIRE datasheet ' + tag + '.csv'
        goodness = goodness.transpose()
        pkl['Shape mode']= pd.Series(IDX)
        pkl['Distance from cluster center'] = pd.Series(IDX_dist)
        for idx,column in enumerate(goodness):
            pkl['probability of shape mode '+str(idx)] = column
        pkl.to_csv(os.path.join(setpath, datasheet), index=False)
    except:
        print('error')
        continue

In [ ]:
# update_csv(IDX, IDX_dist, tag[setidx], setpath, goodness=goodness)